## Tutorial #3: Referencing EEG Data
This tutorial is about effects of different approaches for referencing EEG data.

##### Referencing:
EEG recordings are collected by measuring the voltage differences between a part of the brain and the reference point. After collecting the data, the referernce can be changed and this process is called re-referencing. If the reference changes, the amplitude of the signal at the channels will differ too. Therefore referencing is very critical for EEG data. Rereferencing can be formulazied as follows: 

new electrical potential at a channel = electrical potential at a channel + electrical potential at the original reference - electrical potential at the new reference

##### Selecting the reference:
Athough there is no optimal strategy for selecting the reference, there are several commonly applied approaches. 
One way is to compute the average of potentials at the channels and use this potential as the reference potential. After subtracting this value from electrical potential at each channel, referencing would be completed. 
Another approach is to select the channels where electrical activity is the lowest such as ears. 

Apart from the traditional approaches above, there are several statistical approaches to select the reference that perfom the best. 

In [3]:
import mne
import mne.viz

In [2]:
#Load epoched data
data_file = '../../817_1_PDDys_ODDBALL_Clean_curated-epo'

# Read the EEG epochs:
epochs = mne.read_epochs(data_file + '.fif')

Reading ../../817_1_PDDys_ODDBALL_Clean_curated-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
189 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
